In [105]:
import os
import gzip
import pickle
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error


### **Paso 1: Preprocese los datos**
 - Cree la columna 'Age' a partir de la columna 'Year'.
   Asuma que el año actual es 2021.
 - Elimine las columnas 'Year' y 'Car_Name'.

In [106]:
def preprocess_data(filepath, current_year=2021):
    data = pd.read_csv(filepath, index_col=False, compression="zip")
    data['Age'] = current_year - data['Year']
    data.drop(columns=['Year', 'Car_Name'], inplace=True)
    return data

train_path = "../files/input/train_data.csv.zip"
test_path = "../files/input/test_data.csv.zip"

train_data = preprocess_data(train_path)
test_data = preprocess_data(test_path)

In [107]:
train_data.head()

,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,7.40,8.500,15059,Petrol,Dealer,Automatic,0,5
1,4.00,4.600,30000,Petrol,Dealer,Manual,0,8
2,0.50,0.826,6000,Petrol,Individual,Manual,0,10
3,3.15,4.430,15000,Petrol,Dealer,Manual,0,5
4,1.25,1.500,15000,Petrol,Individual,Manual,0,8


In [108]:
test_data.head()

,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,4.75,9.54,43000,Diesel,Dealer,Manual,0,8
1,7.25,9.85,6900,Petrol,Dealer,Manual,0,4
2,2.85,4.15,5200,Petrol,Dealer,Manual,0,10
3,6.75,8.12,18796,Petrol,Dealer,Manual,0,6
4,6.50,8.61,33429,Diesel,Dealer,Manual,0,6


### **Paso 2: Divida los datasets en x_train, y_train, x_test, y_test.**

In [109]:
x_train = train_data.drop(columns=['Present_Price'])
y_train = train_data['Present_Price']

x_test = test_data.drop(columns=['Present_Price'])
y_test = test_data['Present_Price']

#### **Paso 3: Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:**
- Transforma las variables categoricas usando el método one-hot-encoding.
- Escala las variables numéricas al intervalo [0, 1].
- Selecciona las K mejores entradas.
- Ajusta un modelo de regresion lineal.

In [110]:
colc = ["Fuel_Type", "Selling_type", "Transmission"]
numc = [col for col in x_train.columns if col not in colc]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numc),
        ('cat', OneHotEncoder(handle_unknown="ignore"), colc)
    ],
    remainder=MinMaxScaler(),
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', SelectKBest(score_func=f_regression)),
    ('model', LinearRegression())
])

# Precisión:

pipeline.fit(x_train, y_train)
print("Precisión", pipeline.score(x_test, y_test))

Precisión 0.6663619343263639


### **Paso 4: Optimice los hiperparametros del pipeline usando validación cruzada. Use 10 splits para la validación cruzada. Use el error medio absoluto para medir el desempeño modelo.**

In [111]:
param_grid = {
    'feature_selection__k': range(1, 20),
}

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=10,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder=MinMaxScaler(),
                                                          transformers=[('num',
                                                                         MinMaxScaler(),
                                                                         ['Selling_Price',
                                                                          'Driven_kms',
                                                                          'Owner',
                                                                          'Age']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Fuel_Type',
                                                                          'Selling_type',
                                                                          'Transmission'])])),
                                       ('feature_selection',
                                        SelectKBest(score_func=<function f_regression at 0x0000018DD71CCE50>)),
                                       ('model', LinearRegression())]),
             n_jobs=-1, param_grid={'feature_selection__k': range(1, 20)},
             scoring='neg_mean_absolute_error')

#### **Paso 5: Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz". Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.**






In [112]:
model_path = "../files/models/model.pkl.gz"
os.makedirs(os.path.dirname(model_path), exist_ok=True)
with gzip.open(model_path, 'wb') as f:
    pickle.dump(grid_search, f)

#### **Paso 6: Calcule las metricas r2, error cuadratico medio, y error absoluto medio para los conjuntos de entrenamiento y prueba. Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba.**

In [113]:
def calcular_metricas(y_true, y_pred, dataset_name):
    """Calcula las métricas clave para el conjunto de datos."""
    return {
        "type": "metrics",
        "dataset": dataset_name,
        "r2": float(r2_score(y_true, y_pred)),
        "mse": float(mean_squared_error(y_true, y_pred)),
        "mad": float(median_absolute_error(y_true, y_pred))
    }

def guardar_metricas(metrics, filepath):
    """Guarda las métricas en un archivo JSON línea por línea."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w') as f:
        for metric in metrics:
            f.write(json.dumps(metric) + "\n")

# Generar predicciones
y_train_pred = grid_search.best_estimator_.predict(x_train)
y_test_pred = grid_search.best_estimator_.predict(x_test)

# Calcular métricas
metrics_train = calcular_metricas(y_train, y_train_pred, "train")
metrics_test = calcular_metricas(y_test, y_test_pred, "test")

# Guardar las métricas
output_path = "../files/output/metrics.json"
guardar_metricas([metrics_train, metrics_test], output_path)
